In [9]:
import torch

from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import GAT, GCNConv, to_hetero, SAGEConv

dataset = OGB_MAG(root='./data', preprocess='metapath2vec')
data = dataset[0]

import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero

import torch
from torch.nn import Linear, BatchNorm1d, ModuleList
import torch.nn.functional as F

from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import from_networkx, to_networkx
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch_geometric.nn import GAT, GCNConv, to_hetero, SAGEConv
import torch_geometric.transforms as T

from torch_geometric.nn.conv import HeteroConv

import mlflow.pytorch

from tqdm import tqdm

mlflow.set_tracking_uri("http://localhost:5000")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)


dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=4, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

def count_parameters(model):
    for p in model.parameters():
        print(p.shape)
        print(p)

    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(data)
def train():
    print("training")
    with torch.no_grad():
        model.forward(data.x_dict, data.edge_index_dict)
        print(f"Number of parameters: {count_parameters(model)}")


    optimizer = torch.optim.SGD(model.parameters(), 
                                    lr=0.1,
                                    momentum=0.8,
                                    weight_decay=5e-4)

    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['paper'].train_mask
    loss = F.cross_entropy(out['paper'][mask], data['paper'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

train()

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)
training
torch.Size([4, 128])
Parameter containing:
tensor([[ 0.0676,  0.0734, -0.0207,  0.0812, -0.0194,  0.0178, -0.0430,  0.0519,
          0.0779, -0.0648,  0.0768,  0.0165,  0.0653,  0.0120,  0.0426, -0.0125,
          0.0681,  0.0131, -0.0413,  0.0225, -0.0407, -0.0104, -0.0359,  0.0586,
 

6.316248893737793

In [ ]:
from torch_geometric.nn import GATConv, Linear, to_hetero

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x


model = GAT(hidden_channels=64, out_channels=10)
# model = to_hetero(model, samples[0].metadata(), aggr='sum', debug=True)

try: 
    model = to_hetero(model, samples[0].metadata(), aggr='sum', debug=True)
except NotImplementedError as e:
    print(e)

model

NameError: name 'samples' is not defined

In [ ]:
data = dataset[0]
data

In [ ]:
import torch
from torch.nn import Linear, BatchNorm1d, ModuleList
import torch.nn.functional as F

from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import from_networkx, to_networkx
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch_geometric.nn import GAT, GCNConv, to_hetero, SAGEConv
import torch_geometric.transforms as T

from torch_geometric.nn.conv import HeteroConv

import mlflow.pytorch

from tqdm import tqdm

mlflow.set_tracking_uri("http://localhost:5000")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

5.996389389038086